# Custom cifar-10 conv net with Caffe in Python (Pycaffe)

**Here, I train a custom convnet on the cifar-10 dataset. I did not try to implement any specific known architecture, but to design a new one for learning purposes. It is inspired from the official caffe python ".ipynb" examples available at: https://github.com/BVLC/caffe/tree/master/examples, but not the cifar-10 example itself that was in C++.**

## Dynamically download and convert the cifar-10 dataset to Caffe's HDF5 format using code of another git repo of mine.
More info on the dataset can be found at http://www.cs.toronto.edu/~kriz/cifar.html.

In [1]:
%%time

!rm download-and-convert-cifar-10.py
print("Getting the download script...")
!wget https://raw.githubusercontent.com/guillaume-chevalier/caffe-cifar-10-and-cifar-100-datasets-preprocessed-to-HDF5/master/download-and-convert-cifar-10.py
print("Downloaded script. Will execute to download and convert the cifar-10 dataset:")
!python download-and-convert-cifar-10.py

Getting the download script...
wget: /root/anaconda2/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /root/anaconda2/lib/libssl.so.1.0.0: no version information available (required by wget)
--2015-12-26 18:49:18--  https://raw.githubusercontent.com/guillaume-chevalier/caffe-cifar-10-and-cifar-100-datasets-preprocessed-to-HDF5/master/download-and-convert-cifar-10.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.27.76.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.27.76.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3336 (3.3K) [text/plain]
Saving to: ‘download-and-convert-cifar-10.py’

100%[======================================>] 3,336       --.-K/s   in 0s      

2015-12-26 18:49:19 (1.05 GB/s) - ‘download-and-convert-cifar-10.py’ saved [3336/3336]

Downloaded script. Will execute to download and convert the cifar-10 dataset:

Downloading...
wget: /root/anaconda2/lib/

## Build the model with Caffe. 

In [2]:
import numpy as np

import caffe
from caffe import layers as L
from caffe import params as P

In [3]:
def cnn(hdf5, batch_size):
    n = caffe.NetSpec()
    n.data, n.label = L.HDF5Data(batch_size=batch_size, source=hdf5, ntop=2)
    
    n.conv1 = L.Convolution(n.data, kernel_size=3, num_output=32, weight_filler=dict(type='xavier'))
    n.pool1 = L.Pooling(n.conv1, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.relu1 = L.ReLU(n.pool1, in_place=True)
    n.conv2 = L.Convolution(n.relu1, kernel_size=5, num_output=42, weight_filler=dict(type='xavier'))
    n.pool2 = L.Pooling(n.conv2, kernel_size=2, stride=2, pool=P.Pooling.AVE)
    n.relu2 = L.ReLU(n.pool2, in_place=True)
    n.conv3 = L.Convolution(n.relu2, kernel_size=5, num_output=64, weight_filler=dict(type='xavier'))
    n.sig1 = L.Sigmoid(n.conv3, in_place=True)
    
    n.ip1 = L.InnerProduct(n.sig1, num_output=512, weight_filler=dict(type='xavier'))
    n.sig2 = L.Sigmoid(n.ip1, in_place=True)
    n.ip2 = L.InnerProduct(n.sig2, num_output=10, weight_filler=dict(type='xavier'))
    
    n.accuracy = L.Accuracy(n.ip2, n.label)
    n.loss = L.SoftmaxWithLoss(n.ip2, n.label)
    return n.to_proto()
    
with open('cnn_train.prototxt', 'w') as f:
    f.write(str(cnn('cifar_10_caffe_hdf5/train.txt', 100)))
    
with open('cnn_test.prototxt', 'w') as f:
    f.write(str(cnn('cifar_10_caffe_hdf5/test.txt', 120)))

## Load and visualise the untrained network's internal structure and shape
The network's structure (graph) visualisation tool of caffe is broken in the current release. We will simply print here the data shapes. 

In [4]:
caffe.set_mode_gpu()
solver = caffe.get_solver('cnn_solver_rms.prototxt')

In [5]:
print("Layers' features:")
[(k, v.data.shape) for k, v in solver.net.blobs.items()]

Layers' features:


[('data', (100, 3, 32, 32)),
 ('label', (100,)),
 ('label_data_1_split_0', (100,)),
 ('label_data_1_split_1', (100,)),
 ('conv1', (100, 32, 30, 30)),
 ('pool1', (100, 32, 15, 15)),
 ('conv2', (100, 42, 11, 11)),
 ('pool2', (100, 42, 6, 6)),
 ('conv3', (100, 64, 2, 2)),
 ('ip1', (100, 512)),
 ('ip2', (100, 10)),
 ('ip2_ip2_0_split_0', (100, 10)),
 ('ip2_ip2_0_split_1', (100, 10)),
 ('accuracy', ()),
 ('loss', ())]

In [6]:
print("Parameters and shape:")
[(k, v[0].data.shape) for k, v in solver.net.params.items()]

Parameters and shape:


[('conv1', (32, 3, 3, 3)),
 ('conv2', (42, 32, 5, 5)),
 ('conv3', (64, 42, 5, 5)),
 ('ip1', (512, 256)),
 ('ip2', (10, 512))]

## Solver's params

The solver's params for the created net are defined in a `.prototxt` file. 

Notice that because `max_iter: 100000`, the training will loop 2 times on the 50000 training data. Because we train data by minibatches of 100 as defined above when creating the net, there will be a total of `100000*100/50000 = 200` epochs on some of those pre-shuffled 100 images minibatches.

We will test the net on `test_iter: 100` different test images at each `test_interval: 1000` images trained. 
____

Here, **RMSProp** is used, it is SDG-based, it converges faster than a pure SGD and it is robust.
____

In [7]:
!cat cnn_solver_rms.prototxt

train_net: "cnn_train.prototxt"
test_net: "cnn_test.prototxt"

test_iter: 100
test_interval: 1000

base_lr: 0.0007
momentum: 0.0
weight_decay: 0.005

lr_policy: "inv"
gamma: 0.0001
power: 0.75

display: 100

max_iter: 100000

snapshot: 25000
snapshot_prefix: "cnn_snapshot"
solver_mode: GPU

type: "RMSProp"
rms_decay: 0.98


## Alternative way to train directly in Python
Since a recent update, there is no output in python by default, which is bad for debugging. 
Skip this cell and train with the second method shown below if needed. It is commented out in case you just chain some `shift+enter` ipython shortcuts. 

In [8]:
# %%time
# solver.solve()

## Train by calling caffe in command line
Just set the parameters correctly. Be sure that the notebook is at the root of the ipython notebook server. 
You can run this in an external terminal if you open it in the notebook's directory. 

It is also possible to finetune an existing net with a different solver or different data. Here I do it, because I feel the net could better fit the data. 

In [9]:
%%time
!$CAFFE_ROOT/build/tools/caffe train -solver cnn_solver_rms.prototxt

/root/caffe/build/tools/caffe: /root/anaconda2/lib/liblzma.so.5: no version information available (required by /usr/lib/x86_64-linux-gnu/libunwind.so.8)
I1226 20:49:03.399009  6129 caffe.cpp:184] Using GPUs 0
I1226 20:49:03.610534  6129 solver.cpp:48] Initializing solver from parameters: 
train_net: "cnn_train.prototxt"
test_net: "cnn_test.prototxt"
test_iter: 100
test_interval: 1000
base_lr: 0.0007
display: 100
max_iter: 100000
lr_policy: "inv"
gamma: 0.0001
power: 0.75
momentum: 0
weight_decay: 0.005
snapshot: 25000
snapshot_prefix: "cnn_snapshot"
solver_mode: GPU
device_id: 0
rms_decay: 0.98
type: "RMSProp"
I1226 20:49:03.610752  6129 solver.cpp:81] Creating training net from train_net file: cnn_train.prototxt
I1226 20:49:03.611078  6129 net.cpp:49] Initializing net from parameters: 
state {
  phase: TRAIN
}
layer {
  name: "data"
  type: "HDF5Data"
  top: "data"
  top: "label"
  hdf5_data_param {
    source: "cifar_10_caffe_hdf5/train.txt"
    batch_size: 100
  }
}
layer {
  name: 

## Test the model completely on test data
Let's test directly in command-line:

In [10]:
%%time
!$CAFFE_ROOT/build/tools/caffe test -model cnn_test.prototxt -weights cnn_snapshot_iter_100000.caffemodel -iterations 83

/root/caffe/build/tools/caffe: /root/anaconda2/lib/liblzma.so.5: no version information available (required by /usr/lib/x86_64-linux-gnu/libunwind.so.8)
I1227 00:04:25.706575 24337 caffe.cpp:234] Use CPU.
I1227 00:04:25.928380 24337 net.cpp:49] Initializing net from parameters: 
state {
  phase: TEST
}
layer {
  name: "data"
  type: "HDF5Data"
  top: "data"
  top: "label"
  hdf5_data_param {
    source: "cifar_10_caffe_hdf5/test.txt"
    batch_size: 120
  }
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  convolution_param {
    num_output: 32
    kernel_size: 3
    weight_filler {
      type: "xavier"
    }
  }
}
layer {
  name: "pool1"
  type: "Pooling"
  bottom: "conv1"
  top: "pool1"
  pooling_param {
    pool: MAX
    kernel_size: 2
    stride: 2
  }
}
layer {
  name: "relu1"
  type: "ReLU"
  bottom: "pool1"
  top: "pool1"
}
layer {
  name: "conv2"
  type: "Convolution"
  bottom: "pool1"
  top: "conv2"
  convolution_param {
    num_output: 42
    k

### 71% accuracy
Coffe brewed. The net for sure could be fine tuned some more with better solver parameters. 

Let's convert the notebook to github markdown:

In [11]:
!jupyter nbconvert --to markdown custom-cifar-10.ipynb 
!mv custom-cifar-10.md README.md

[NbConvertApp] Converting notebook custom-cifar-10.ipynb to markdown
[NbConvertApp] Writing 404667 bytes to custom-cifar-10.md
